In [1]:
import numpy as np

#countsf = np.core.records.fromfile('1983dec31.f06', formats='i1,i1,i2,i2,i2,(20,)i2,(20,)i2,i4,(9,)i4', names='one, two, three, four, five, six, seven, eight, nine', byteorder='<')

In [2]:
#data from DMSP f13, ssj4 instrument, January 2000
def Read_data(file_name):
    recarray = np.core.records.fromfile(file_name, formats='i2,i2,i2,i2,i2,i2,i2,i2,i2,i2,i2,i2,i2,i2,i2,(2580,)i2,i2,(44,)i2', 
                                   names = '''day,hour,minute,second,year,geo_lat,geo_long,alt,lat_110,long_110,corrected_mlat,
                                   corrected_mlong,mlt_hour,mlt_minute,mlt_second,channels,indicator,zero_fill''',byteorder='>')
    return recarray
counts1 = Read_data('j4f1300001')
counts2 = Read_data('j4f1300002')

In [3]:
data = np.zeros((43,(len(counts1['channels'])),60),dtype = 'int16')
i = 0
for i in range(0,43):
    data[i] += counts1['channels'][:,i:len(counts1['channels'][0]):43]
    i += 1
#print(data)

In [4]:
#One of the data pieces at the beginning of each minute of data is magnetic latitude, corrected for an altiude of 110km.
#This is the value I want to filter the data by to limit my analysis to the cusp region. There is a conversion listed in the 
#documentation for getting the value in degrees. The following loop performs this conversion.
mlat = np.zeros(len(counts1['corrected_mlat']))
i = 0
for i in range(len(counts1['corrected_mlat'])):
    if counts1['corrected_mlat'][i] > 1800:
        mlat[i] += float((counts1['corrected_mlat'][i]-4995))/10.0
    else:
        mlat[i] += float((counts1['corrected_mlat'][i]-900))/10.0
    i += 1
print(mlat)
#The magnetic latitude is only given for each minute, but we have data for each second. Therefore I am going to create a loop
#that repeats the last entry of magnetic latitude if the "minute" data entry is still the same. I can't just repeat the numbers
#60 times each because the data is incomplete. This will match the dimensions of the magnetic latitude array to the data sets.

#extended_mlat = np.zeros(len(data[1]))
#i = 0 
#j = 0
#k = 0
#for i in range(len(mlat)):
#    extended_mlat[j] += mlat[i]
#    j+=1
 #   if data[1,k] - data[1,k+1] = 0:
 #       extended_mlat[j] += mlat[i]
 #       k+=1
 #       j+=1
 #   i+=1

#unique, counts = np.unique(data[1], return_counts=True)
#print(counts)
#sum(counts)

[66.3 63.5 60.7 ... 23.7 25.9 28.3]


In [5]:
#taking the polar cusp region to have a bounary of 76-82 degrees, the next loops will filter the data for this region by adding
#the data to a new list if the magnetic latitude matches this region for that second of data.

#new lists
#ch_cusp = {}
#for a in range (0,43):
#    ch_cusp[a] = []

#filtering loop
#i = 0
#j = 0
#for j in range(0,43):
#    for i in range(len(mag_lat_extended)): 
#        if mag_lat_extended[i] >= 76 and mag_lat_extended[i] <= 82:
#            ch_cusp[j].append(ch[j][i])
#        i+=1
#    j+=1

In [6]:
#this loop converts the filtered data's timestamps from miliseconds to seconds, and non-negative values
#i = 0
#true_seconds = []
#for i in range(len(ch_cusp[2])):
#    value = ch[2][i]/1000
#    if value < 0:
#        value = value + 65
#    i+=1
#    true_seconds.append(value)